# Fairness, Accountability, Transparency and Ethics Course


## Universitat Pompeu Fabra (UPF)
### 20/21

### Author: David Solans (david.solans@upf.edu)

**Legend** <br>
In this notebook we use:    
<div class="alert alert-block col-md-7 alert-info">To recall information from the theory classes and other tips</div>
<div class="alert alert-block col-md-7 alert-warning">To point important things that should not be 
   forgotten</div> 
<div class="alert alert-block col-md-7 alert-success">To indicate tasks to be done by students</div>

# 3. Algorithmic Fairness with Python and IBM AIF360

###  Libraries used in this notebook
You will need to install: **numpy**, **pandas**, **matplotlib**, and **sklearn**.

In [1]:
## Required statements
import pandas as pd
import numpy as np

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult

from aif360.metrics import BinaryLabelDatasetMetric

from IPython.display import Markdown, display


from IPython.display import Markdown, display
import matplotlib.pyplot as plt
%matplotlib inline

# Session 05

## 3.0 Introduction
<br>
<div class="alert alert-block alert-info col-md-12">
<p>In previous Lab we were using pandas to load the Adult dataset (See Lab2 notebook).<p>
<p>In pandas we would use the following statements to load a dataset:</p>

</div>






In [2]:
columns_names = (
    'age',
    'workclass', #Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
    'fnlwgt', # "weight" of that person in the dataset (i.e. how many people does that person represent) -> https://www.kansascityfed.org/research/datamuseum/cps/coreinfo/keyconcepts/weights
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income',
)

df = pd.read_csv("../Data/adult.data", names=columns_names)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## 3.1 AIF360 


AIF360 is a Python library developed by IBM that aims to help ML practicioners to include Algorithmic Fairness considerations on their developments.

It's one of the most used libraries in the topic of Algorithmic Fairness and little by little is becoming an standard.

Documentation is available [here](https://aif360.readthedocs.io/en/latest/)

Video tutorial is available [here](https://www.youtube.com/watch?v=X1NsrcaRQTE)


AIF360 offers tools for **fairness metrics** and **bias mitigation algorithms**

## 3.2 Using AIF360

### 1. Installation

In [3]:
## Uncomment this line to install the last version of the library
#!pip install -U aif360

In AIF360, there is the class Dataset that can be used for similar purposes. AIF360 already has some pre-loaded datasets. 

In the following example, we will use again the Adult Census dataset.

### 2. Loading Common Datasets: pre loaded in the tool


Last versions of the libary do not install the preloaded datasets directly but require a manual installation of them. 

You can see [here](https://aif360.readthedocs.io/en/latest/modules/datasets.html#common-datasets) what are the **Common Datasets** that you can directly download.

You can use the following script to automatize the downloading of the three main datasets:


In [4]:
## Uncomment this to automatize the downloading of AIF pre-loaded datasets
#!sh download_aif360_datasets.sh

### 3. Data loading: Adult Census Income dataset

Info about the AdultDataset in AIF360 is available [here](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.AdultDataset.html#aif360.datasets.AdultDataset).

The following code specifies the privileged and unprivileges groups.


As most of the available algorithmic fairness tools, AIF360 needs you to first specify the population groupings to be used for the fairness assessment.

In [5]:
## Import dataset

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = load_preproc_data_adult(['sex'])
   

#random seed for calibrated equal odds prediction
np.random.seed(1)


In [6]:
## Printing the content
dataset_orig

               instance weights features                                      \
                                         protected attribute                   
                                    race                 sex Age (decade)=10   
instance names                                                                 
0                           1.0      0.0                 1.0             0.0   
1                           1.0      1.0                 1.0             0.0   
2                           1.0      1.0                 1.0             0.0   
3                           1.0      0.0                 1.0             0.0   
4                           1.0      1.0                 0.0             1.0   
...                         ...      ...                 ...             ...   
48837                       1.0      1.0                 0.0             0.0   
48838                       1.0      1.0                 1.0             0.0   
48839                       1.0      1.0

### 4. Printing characteristics of the original dataset

In [7]:
# print out some labels, names, etc.
display(Markdown("#### Dataset shape"))
print(dataset_orig.features.shape)

display(Markdown("#### Dataset label name"))
print(dataset_orig.label_names)

display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig.favorable_label, dataset_orig.unfavorable_label)

display(Markdown("#### Protected attribute names"))
print(dataset_orig.protected_attribute_names)

display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig.privileged_protected_attributes, 
      dataset_orig.unprivileged_protected_attributes)

display(Markdown("#### Dataset feature names"))
print(dataset_orig.feature_names)

#### Dataset shape

(48842, 18)


#### Dataset label name

['Income Binary']


#### Favorable and unfavorable labels

...

['sex']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


### 5. Computing fairness metric on the original dataset

The Dataset corresponds to a BinarLabelDataset instance.

The tool contains different [metrics](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.BinaryLabelDatasetMetric.html#aif360.metrics.BinaryLabelDatasetMetric) to assess if there are significant disparities across groups in the input data.

In [8]:
metric_orig = BinaryLabelDatasetMetric(dataset_orig, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

display(Markdown("#### Original dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig.mean_difference())


#### Original dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.194516


### 6. Splitting the dataset between train and test

Datasets contain an split function that can be used to divide the data between train and test sets.

In [9]:
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)


### 7.Computing fairness metric on the TRAINING dataset

We can again test whe mean difference in the training set.

In [10]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

display(Markdown("#### Training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.190244


### 6. Train a classifier using the training data

Preparing data for scikit-learn.

In [18]:
from sklearn import svm
# Logistic regression classifier and predictions
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()


### Exercise 1 (1 point):

<br>
<div class="alert alert-block alert-success col-md-12">

<p>Use the data above to train three different scikit-learn classifiers of your election and store them in an array</p>

</div>


In [27]:
## Your code here
#clfs = []

### 7. Predicting classification for the test set

Obtaining predictions

### Exercise 2 (1 point):

<br>
<div class="alert alert-block alert-success col-md-12">

<p>Get the predictions for our X_test for each of the classifiers. Store the resulting result on an array </p>

</div>


In [16]:
## Your code_here
#clfs_preds = []

### 8. Assessing classification fairness for the predictions

In this cell, we will assess different [classification fairness metrics](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.ClassificationMetric.html#aif360.metrics.ClassificationMetric)

In this analysis, we will use:
- **Balanced accuracy.** 
Accuracy balanced across privileged and unprivileged individuals.
$$0.5 * ACC_{D=unprivileged} + 0.5*ACC_{D=privileged} $$


- **Average odds difference.** 
Average of difference in FPR and TPR for unprivileged and privileged groups:
$$\frac{1}{2}[(FPR_{D=unprivileged} - FPR_{D=privileged}) + (TPR_{D=unprivileged}-TPR_{D=unprivileged})]$$

- **Disparate impact.** 
Ratio of probabilities of favorable label predicted for each group:
$$\frac{P(\hat Y = 1 | D=unprivileged)}{P(\hat Y = 1 | D=privileged)}$$

- **Error rate ratio.** Ratio of error rate for across groups:
$$\frac{ERR_{D=unprivileged}}{ERR_{D=privileged}}$$

### Exercise 3 (1 point):

<br>
<div class="alert alert-block alert-success col-md-12">

<p>Using the prediction results seen above, obtain the values for each of the fairness metrics mentioned above and store them in an array</p>

</div>

In [24]:
## Your code here
#clfs_preds_fairness = []

### 9. Plotting and interpretation of the obtained results

### Exercise 4 (1 point):

<br>
<div class="alert alert-block alert-success col-md-12">

<p>Plot the obtained values for the classification metrics for each of the trained classifiers</p>

</div>

In [26]:
## Your code here

### Exercise 5 (1 point):

<br>
<div class="alert alert-block alert-success col-md-12">

<p>Provide an interpretation of the observed results (max 5 lines)</p>

</div>

<div class="alert alert-block alert-info col-md-12">

<p>Focusing in the two metrics for which you observe bigger and smaller differences is enough.</p>

</div>

**Your answer here**:

# Session 06

In this session, you will use AIF360 yourselves.

### 10. Unfairness mitigation algorithms  in AIF360
The library contains a set of algorithms to mitigate potential unfairness in machine-learning pipelines.

The provided algorithms correspond implementation of to state of the art techniques (most of them contain a link to a scientific paper that introduced them).

The existing algorithms can be found [here](https://aif360.readthedocs.io/en/latest/modules/algorithms.html).

They are divided between:
- **Preprocessing.** Algorithms that are applied before starting the learning process (modification of the input data before feeding it to the model).

- **Inprocessing.** Algorithms that are applied during the learning process (e.g.: modification of the loss function to account for fairness).

- **Postprocessing.** Algorithms that are applied after the predictions of the models, substituting them for more fair solutions.


### Exercise 6 (1 point):

<br>
<div class="alert alert-block alert-success col-md-12">

<p>Load one of the preloaded datasets in AIF360 different of the adult dataset and plot the characteristics of it</p>

</div>

<div class="alert alert-block alert-info col-md-12">

<p>The existing datasets can be loaded with different sensitive attributes</p>
<p>You can add necessary imports for loading the dataset</p>

</div>

In [29]:
## Your code here
## aif_dataset = 

### Exercise 7 (0.5 point):
<br>
<div class="alert alert-block alert-success col-md-12">

<p>Divide your data between train and test</p>

</div>

In [28]:
## Your code here

### Exercise 8 (0.5 point):
<br>
<div class="alert alert-block alert-success col-md-12">

<p>Train a scikit-learn model of your election with the data above</p>

</div>

In [30]:
## Your code here
#clf_orig = 

### Exercise 9 (0.5 point):
<br>
<div class="alert alert-block alert-success col-md-12">

<p>Assess the performance from the fairness point of view of the created model</p>

</div>

<div class="alert alert-block alert-info col-md-12">

<p>For that, you should plot the result of some fairness metric for classification.</p>
<p> You can just use the same metrics we used above, but remember to add other metrics if you need them to detect changes of performance in the following exercises</p>

</div>

In [ ]:
## Your code here

### Exercise 10 (2.5 points)

<br>
<div class="alert alert-block alert-success col-md-12">
<ul>

<li><p>Apply an unfairness mitigation algorithm of your election, train a new classifier of the same family used above and assess 
the new performance from the fairness point of view</p></li>

<li><p>Provide a textial description of the results (max 3 lines)</p></li>

</ul>

</div>

<div class="alert alert-block alert-info col-md-12">

<p>Although you can try other algorihtms if you want, the suggested algorithms to mitigate unfairness are the following:</p>

    
<ul>
<li><b>Preprocessing</b>
<ul>
<li>Disparate Impact Remover</li>
<li>Rewighing (you need a classifier that allows you to parametrize sample weights for this)</li>
</ul>

</li>
<li><b>Inprocessing</b>
<ul>
<li>MetaFairClassifier</li>
</ul>
</li>
<li><b>Postprocessing</b>
<ul>
<li>Disparate Impact Remover</li>
<li>EqOddsPostprocessing (You need to use the equalized odds ratio to measure performance of this classifier and the classifier trained above)</li>
</ul>
</li>
</ul>

<p>You can add necessary imports for using the algorithm</p>

</div>


<div class="alert alert-block alert-warning col-md-12">
    <p>Sometimes, certain algorithms do not work well with certain classifiers / output distributions</p>
    <p>This can happen for several reasons, one of them being the changes done for subsequent versions of the tool</p>
    <br>
    
    <b>If you see that your unfairness mitigation algorithm does not work as expected, do not invest too much time trying to solve it, go </b>
</div>   

In [ ]:
## Your code here

**Your explaination here**